# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [4]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [5]:
data = pd.read_csv("./../data/customer_product_sales.csv")

In [7]:
data.head(10)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
5,83106,Hope,Bowers,6484922,229,Bread - Hot Dog Buns,22
6,11253,Nicolas,Holmes,4878478,229,Bread - Hot Dog Buns,3
7,35107,Judith,Miranda,2353899,229,Bread - Hot Dog Buns,9
8,15088,Ruby,Lowe,37021,229,Bread - Hot Dog Buns,4
9,26031,Josephine,Curry,2327232,229,Bread - Hot Dog Buns,7


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [9]:
df = pd.DataFrame(data)
df2 = df.groupby(['CustomerID','ProductName'])[['Quantity']].sum()
df2

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [12]:
customer_product = pd.pivot_table(data, values='Quantity', index='ProductName',
                           columns='CustomerID',aggfunc=np.sum, fill_value=0)


customer_product

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [10]:
distance = pd.DataFrame(1/(1 + squareform(pdist(customer_product.T, 'euclidean'))),
                       index=customer_product.columns, columns=customer_product.columns)
distance

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [13]:
similar_customer = distance[412].sort_values(ascending=False)[1:6] #SOLO QUEREMOS VER 5 RESULTADOS PARA UN CLIENTE DETERMINADO
similar_customer

CustomerID
3317    0.085638
3535    0.084959
1008    0.083333
1072    0.082709
639     0.082403
Name: 412, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [14]:
records_customers = df2.loc[similar_customer.index]
records_customers

Quantity
CustomerID ProductName                          
3317       Bay Leaf                            1
           Beef - Ground Medium                1
           Beef - Rib Eye Aaa                  1
           Butter - Unsalted                   1
           Cheese - Victor Et Berthold         1
...                                          ...
639        Veal - Inside, Choice               1
           Water, Tap                          1
           Wine - Magnotta - Belpaese          1
           Wine - Toasted Head                 2
           Wine - Two Oceans Cabernet          1

[272 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [19]:
aggregate_product = records_customers.groupby('ProductName').sum().sort_values(by= "Quantity", ascending=False)
aggregate_product

,Quantity
ProductName,
Olive - Spread Tapenade,3
Sprouts - Baby Pea Tendrils,3
Wine - Blue Nun Qualitatswein,3
"Veal - Inside, Choice",3
"Pepper - Black, Whole",3
...,...
Flour - Whole Wheat,1
Foam Cup 6 Oz,1
Garlic,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [21]:
merge_table = customer_product.merge(aggregate_product, left_on='ProductName', right_on='ProductName')
merge_table = merge_table[merge_table[412]==0]
merge_table = merge_table.reset_index()
merge_table

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
0,Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,...,25,0,0,0,0,0,0,0,0,2
1,"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,...,25,0,0,0,0,0,0,0,0,1
2,Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,1
3,Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,...,0,0,0,25,25,25,0,25,0,1
4,Assorted Desserts,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,25,0,0,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,"Wine - White, Colubia Cresh",0,1,0,0,0,1,0,0,0,...,0,0,0,25,0,0,0,0,0,1
173,Wonton Wrappers,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
174,Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,...,0,0,25,0,0,0,0,0,0,1
175,Yoghurt Tubes,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,25,0,0,2


In [22]:
recommended_products = merge_table.sort_values(by= 'Quantity', ascending = False)[['ProductName',33,'Quantity']].head(5)
recommended_products

,ProductName,33,Quantity
124,"Sauce - Gravy, Au Jus, Mix",0,3
141,Sprouts - Baby Pea Tendrils,0,3
98,Olive - Spread Tapenade,0,3
163,Wine - Blue Nun Qualitatswein,0,3
31,Cheese - Cambozola,0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [40]:
def recomendaciones_para_usuarios(customerID):
    generar_lista_recomendados = distance[customerID].sort_values(ascending=False)[1:6]
    clientes_similares_seleccionado = df2.loc[generar_lista_recomendados.index]
    agregando_compras_similares = clientes_similares_seleccionado.groupby('ProductName').sum().sort_values(ascending=False)
    tabla_unida = customer_product.merge(agregando_compras_similares, left_on='ProductName', right_on='ProductName')
    tabla_unida = tabla_unida[tabla_unida[customerID]==0]
    tabla_unida = tabla_unida.reset_index()
    productos_posibles = tabla_unida.sort_values(by= 'Quantity', ascending = False)[['ProductName','Quantity']].head(5)
    return productos_posibles
customer_recomendations = {}
customers_ID = list(distance.columns)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [42]:
for i in customers_ID:
    customer_recomendations[i] = recomendaciones_para_usuarios(i)
customer_recomendations

TypeError: sort_values() missing 1 required positional argument: 'by'

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.